In [1]:
import os

import dbf
import pandas as pd
from dbfread import DBF

In [90]:
path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
name_projects = "tmp1"
name_file = r"equip.DBF"
# name_file = r"pgdynobj.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

class TypeDevice:
    P3U30 = 'P3U30'
    P3F30 = 'P3F30'
    P3T32 = 'P3T32'

class Project:
    def __init__(self, name_project=None, name_cluster='Cluster1', name_server='IOServer1'):
        self.path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server


    def write_equip(self):
        cols = ['NAME', 'CLUSTER', 'TYPE', 'IODEVICE', 'TAGPREFIX', 'COMMENT', 'CUSTOM1']
        count = 3
        start = 30
        end = start + count
        params_list = []
        self.type_device_current = self.type_devices.P3U30

        for i in range(start, end):
            io_device = f'Cell_{i}'
            values = [f'{io_device}_{self.type_device_current}', self.cluster, self.type_device_current, io_device, 'Relay/', f'Защита ячейки {i}', f'{io_device}\\']
            params_list.append(dict(zip(cols, values)))

        self.to_dbf(self.name_files['Оборудование'], params_list)


    def write_io(self):
        cols = ['SERVER', 'NAME', 'ADDR', 'PROTOCOL', 'PORT', 'MODE', 'PRIORITY', 'COMMENT']
        count = 3
        start = 30
        end = start + count
        params_list = []
        self.type_device_current = self.type_devices.P3U30

        numb = 0
        with dbf.Table(self.get_path_file(self.name_files['IO']), codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_ONLY)
            numb = int(tb[-1]['NUMBER'])

        for i in range(start, end):
            io_device = f'Cell_{i}'
            numb += 1
            values = {
                'SERVER': self.server,
                'NAME': io_device,
                'NUMBER': f'{numb}',
                'ADDR': r'[RUN]:conf61850.xml',
                'PROTOCOL': 'IEC61850N',
                'PORT': '',
                'MODE': 'Primary',
                'PRIORITY': '1',
                'MEMORY': 'False',
                'COMMENT': ''
            }
            # values = [self.server, io_device, r'[RUN]:conf61850.xml', 'IEC61850N', '', 'Primary', '1', '']
            params_list.append(values)

        self.to_dbf(self.name_files['IO'], params_list)


    def to_dbf(self, name_file, params_list):
        path_file = self.get_path_file(name_file)

        with dbf.Table(path_file, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def get_path_file(self, name_file):
        return os.path.join(self.path_project, name_file)


    def show_equip(self):
        return pd.DataFrame(DBF(self.get_path_file(self.name_files['Оборудование']), encoding='cp1251'))

    def show_tags(self):
        return pd.DataFrame(DBF(self.get_path_file(self.name_files['Теги']), encoding='cp1251'))

    def show_io(self):
        return pd.DataFrame(DBF(self.get_path_file(self.name_files['IO']), encoding='cp1251'))

In [91]:
project = Project('tmp1')

In [92]:
project.write_io()

In [76]:
dd = dbf.Table(os.path.join(path_projects, 'tmp1', 'units.DBF'), codepage='cp1251')

In [77]:
dd.open(mode=dbf.READ_ONLY)

dbf.Table('c:\\ProgramData\\AVEVA\\Citect SCADA 2018 R2\\User\\tmp1\\units.DBF', status=<DbfStatus.READ_ONLY: 1>)

In [78]:
len(dd)

2

In [75]:
dd.close()

In [68]:
project.show_io()

,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [57]:
project = Project('tmp1')
project.write_io()

ValueError: invalid literal for int() with base 10: '        '

Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_39_64.pyx", line 1034, in _pydevd_bundle.pydevd_cython_win32_39_64.PyDBFrame.trace_dispatch
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [20]:
# Удаление строки
rec = table[-1]
dbf.delete(rec)
table.pack()

In [11]:
name_file = r"units.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df


,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [10]:
name_file = r"variable.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\VO3GGIO99\Ind\t,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\t
1,Cell_2\VO3GGIO99\Ind\stVal,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\stVal
2,Cell_2\VO4GGIO100\Ind\stVal,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\stVal
3,Cell_2\VO4GGIO100\Ind\t,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\t
4,Cell_2\Obj2CSWI2\Pos\stVal,STRING,Cell_2,Relay/Obj2CSWI2$ST$Pos$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,Cell_20\U3ppMMXU5\PPV\phsBC\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsBC$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
699,Cell_20\U3ppMMXU5\PPV\phsCA\cVal,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal
700,Cell_20\U3ppMMXU5\PPV\phsCA\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
701,Cell_20\UovMMXU1\PhV\neut\cVal,STRING,Cell_20,Relay/UovMMXU1$MX$PhV$neut$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,UovMMXU1\PhV\neut\cVal


In [7]:
name_file = r"equip.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,Cell_2_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_2\,,,,...,,,,,,,,,,
1,Cell_3_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_3\,,,,...,,,,,,,,,,
2,Cell_4_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_4\,,,,...,,,,,,,,,,
3,Cell_5_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_5\,,,,...,,,,,,,,,,
4,Cell_6_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_6\,,,,...,,,,,,,,,,
5,Cell_7_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_7\,,,,...,,,,,,,,,,
6,Cell_8_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_8\,,,,...,,,,,,,,,,
7,Cell_9_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_9\,,,,...,,,,,,,,,,
8,Cell_10_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_10\,,,,...,,,,,,,,,,
9,Cell_11_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_11\,,,,...,,,,,,,,,,
